In [15]:
import pandas as pd
import re
import torch
from torch import nn
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
train_df = pd.read_csv('train-data.tsv', sep='\t', header=None, names=['target', 'text'])
val_df = pd.read_csv('valid-data.tsv', sep='\t', header=None, names=['target', 'text'])

In [17]:
def normalize_text(txt):
    # lowercase
    txt = txt.lower()
    # remove special characters and digits
    txt = re.sub(r'[^\w\s]', ' ', txt)
    txt = re.sub(r'\s+', ' ', txt)
    return txt


In [18]:
train_df['target'] = train_df['target'].apply(lambda x: 1 if x == 'ham' else 0)
train_df['text'] = train_df['text'].apply(normalize_text)
val_df['target'] = val_df['target'].apply(lambda x: 1 if x == 'ham' else 0)
val_df['text'] = val_df['text'].apply(normalize_text)

In [19]:
encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
x_train = torch.tensor(encoder.encode(train_df['text'].tolist())).to(device)  

x_val = torch.tensor(encoder.encode(val_df['text'].tolist())).to(device)

y_train = torch.tensor(train_df['target'].tolist(), dtype=torch.float32).to(device)
y_val = torch.tensor(val_df['target'].tolist(), dtype=torch.float32).to(device)

In [27]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(768, 256),
            nn.PReLU(),
            nn.Dropout(0.6),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)
            
    

torch.manual_seed(42)
model = Network().to(device)
loss_fn = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

def train_loop(x, y, model, loss_fn, optimizer, x_val = None, y_val = None, epochs = 200):
    for epoch in range(epochs):
        model.train()
        y_pred = model(x).squeeze()
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            if x_val is not None and y_val is not None:
                model.eval()
                y_pred = model(x_val).squeeze()
                loss = loss_fn(y_pred, y_val)
                acc = ((y_pred > 0.5) == y_val).float().mean()
                print(f"Epoch: {epoch + 1}, Loss: {loss.item()}, Validation Accuracy: {100 * acc.item():.2f}%")

In [28]:
train_loop(x_train, y_train, model, loss_fn, optimizer, x_val, y_val, epochs=200)

Epoch: 10, Loss: 0.6173142790794373, Validation Accuracy: 86.57%
Epoch: 20, Loss: 0.33061283826828003, Validation Accuracy: 86.57%
Epoch: 30, Loss: 0.18995530903339386, Validation Accuracy: 93.89%
Epoch: 40, Loss: 0.11712203919887543, Validation Accuracy: 94.97%
Epoch: 50, Loss: 0.06929001212120056, Validation Accuracy: 97.63%
Epoch: 60, Loss: 0.05195258557796478, Validation Accuracy: 98.13%
Epoch: 70, Loss: 0.04525814577937126, Validation Accuracy: 98.64%
Epoch: 80, Loss: 0.040131304413080215, Validation Accuracy: 98.85%
Epoch: 90, Loss: 0.0383618026971817, Validation Accuracy: 98.92%
Epoch: 100, Loss: 0.03871142119169235, Validation Accuracy: 98.92%
Epoch: 110, Loss: 0.039531826972961426, Validation Accuracy: 98.85%
Epoch: 120, Loss: 0.03862057998776436, Validation Accuracy: 98.99%
Epoch: 130, Loss: 0.038612909615039825, Validation Accuracy: 98.92%
Epoch: 140, Loss: 0.03934590518474579, Validation Accuracy: 98.85%
Epoch: 150, Loss: 0.039017535746097565, Validation Accuracy: 98.92%
Ep

In [29]:
def predict_message(msg):
    model.eval()
    with torch.inference_mode():
        x = torch.tensor(encoder.encode([normalize_text(msg)])).to(device)
        y_pred = model(x).squeeze()
        return (y_pred.cpu().detach().item(), 'ham' if y_pred.item() > 0.5 else 'spam')

In [30]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(f"Test with message: '{msg}' failed. Expected: '{ans}', but got '{prediction[1]}'")
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
